### Installing Libraires and Setting up the environment

In [1]:
!pip install datasets
!pip install langdetect
!pip install numpy
!pip install transformers
!pip install scikit-learn
!pip install torch --index-url https://download.pytorch.org/whl/cu124
!pip install torchvision --index-url https://download.pytorch.org/whl/cu124
!pip install torchaudio --index-url https://download.pytorch.org/whl/cu124
!pip install sentencepiece
!pip install protobuf
!pip install sentence-transformers
!pip install faiss-cpu
!pip install evaluate
!pip install tqdm
!pip install colorama

from IPython.display import clear_output
clear_output()

### Importing Libraries

In [2]:
import torch
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast, Trainer, TrainingArguments
from datasets import load_dataset, Dataset
from sentence_transformers import SentenceTransformer
import faiss
from sklearn.model_selection import train_test_split
import numpy as np
import evaluate

D:\Github\Sanvad-Multilingual-Chatbot\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
if torch.cuda.is_available():
    print("CUDA is available!")
    print(f"Number of GPUs: {torch.cuda.device_count()}")
    print(f"CUDA version: {torch.version.cuda}")
    print(f"Current GPU: {torch.cuda.get_device_name(torch.cuda.current_device())}")
else:
    print("CUDA is not available.")

CUDA is available!
Number of GPUs: 1
CUDA version: 12.4
Current GPU: NVIDIA GeForce GTX 1650 Ti


### Load pre-trained model and tokenizer

In [4]:
model_name = "facebook/mbart-large-50-many-to-many-mmt"
tokenizer = MBart50TokenizerFast.from_pretrained(model_name)
model = MBartForConditionalGeneration.from_pretrained(model_name)

D:\Github\Sanvad-Multilingual-Chatbot\venv\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


### Prepare FLORES-101 dataset

In [5]:
def load_and_preprocess_data():
    dataset = load_dataset("facebook/flores", "all", trust_remote_code=True)
    languages = ["eng_Latn", "hin_Deva", "ben_Beng", "tam_Taml", "tel_Telu", "mar_Deva", "urd_Arab", "guj_Gujr", "kan_Knda", "mal_Mlym"]
    data = []
    for lang in languages:
        for split in ["dev", "devtest"]:
            entries = dataset[split][f"sentence_{lang}"]
            for entry in entries:
                data.append({"text": entry, "lang": lang})
    return data

### Tokenize function

In [36]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=128)

### Prepare dataset for training

In [37]:
data = load_and_preprocess_data()
train_data, eval_data = train_test_split(data, test_size=0.1, random_state=42)

train_dataset = Dataset.from_dict({
    "text": [item["text"] for item in train_data],
    "lang": [item["lang"] for item in train_data]
})
eval_dataset = Dataset.from_dict({
    "text": [item["text"] for item in eval_data],
    "lang": [item["lang"] for item in eval_data]
})

tokenized_train = train_dataset.map(tokenize_function, batched=True)
tokenized_eval = eval_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/18081 [00:00<?, ? examples/s]

Map:   0%|          | 0/2009 [00:00<?, ? examples/s]

### Define training arguments, Trainer and train the model

In [53]:
# Function to encode the data
def encode_data(examples):
    model_inputs = tokenizer(examples['lang'], truncation=True, padding="max_length", max_length=256)
    # Assuming we want to create labels for a translation task
    model_inputs['labels'] = examples['input_ids']
    return model_inputs

# Encoding the train and validation datasets
train_encoded = tokenized_train.map(encode_data, batched=True)
eval_encoded = tokenized_eval.map(encode_data, batched=True)

# Prepare datasets for training by setting the format for PyTorch
train_encoded.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])
eval_encoded.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])

# Define training arguments
args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=500,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="accuracy"
)

# Load accuracy metric from evaluate library
metric = evaluate.load('accuracy')

# Compute metrics function to evaluate the model
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)  # Correct axis for softmax output
    return metric.compute(predictions=predictions, references=labels)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_encoded,
    eval_dataset=eval_encoded,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# Start training
trainer.train()

Map:   0%|          | 0/18081 [00:00<?, ? examples/s]

Map:   0%|          | 0/2009 [00:00<?, ? examples/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 978.00 MiB. GPU 

### Save the fine-tuned model

In [ ]:
model.save_pretrained("./fine_tuned_mbart")
tokenizer.save_pretrained("./fine_tuned_mbart")

### Implement RAG

In [ ]:
sentence_model = SentenceTransformer('distiluse-base-multilingual-cased-v2')

def create_faiss_index(knowledge_base):
    embeddings = sentence_model.encode([item['text'] for item in knowledge_base])
    dimension = embeddings.shape[1]
    index = faiss.IndexFlatL2(dimension)
    index.add(embeddings.astype('float32'))
    return index

faiss_index = create_faiss_index(data)

def retrieve_passages(query, top_k=5):
    query_embedding = sentence_model.encode([query])
    _, indices = faiss_index.search(query_embedding.astype('float32'), top_k)
    return [data[idx] for idx in indices[0]]

def generate_response(query, retrieved_passages, src_lang, tgt_lang):
    context = " ".join([passage['text'] for passage in retrieved_passages])
    input_text = f"Context: {context}\nQuery: {query}\nResponse:"

    tokenizer.src_lang = src_lang
    inputs = tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True)

    outputs = model.generate(
        **inputs,
        forced_bos_token_id=tokenizer.lang_code_to_id[tgt_lang],
        max_length=150,
        num_beams=5,
        no_repeat_ngram_size=2
    )

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

### Main RAG function

In [ ]:
def rag_chatbot(query, src_lang, tgt_lang):
    try:
        retrieved_passages = retrieve_passages(query)
        response = generate_response(query, retrieved_passages, src_lang, tgt_lang)
        return response
    except Exception as e:
        print(f"Error in RAG chatbot: {str(e)}")
        return f"An error occurred while processing your query. Please try again."

### Interactive chat interface

In [ ]:
print("Welcome to the Multilingual Chatbot!")
print("Available languages: eng_Latn, hin_Deva, ben_Beng, tam_Taml, tel_Telu, mar_Deva, urd_Arab, guj_Gujr, kan_Knda, mal_Mlym")

while True:
    query = input("Enter your query (or 'quit' to exit): ")
    if query.lower() == 'quit':
        break

    src_lang = input("Enter the source language code: ")
    tgt_lang = input("Enter the target language code: ")

    response = rag_chatbot(query, src_lang, tgt_lang)
    print(f"Response: {response}\n")